<a href="https://colab.research.google.com/github/good0neighbour/ESTCampAI/blob/main/%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C/ex02_%EC%98%81%ED%99%94_%EC%B6%94%EC%B2%9C_%EA%B8%B0%EB%8A%A5_%EA%B5%AC%ED%98%84_Collaborative_Filtering(%ED%98%91%EC%97%85_%ED%95%84%ED%84%B0%EB%A7%81).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/ESTCampAI/추천시스템

/content/drive/MyDrive/ESTCampAI/추천시스템


### 콘텐츠 기반 필터링의 한계와 협업 필터링
- 콘텐츠 기반 필터링의 한계점 극복
  - 콘텐츠 기반 필터링은 아이템 내재적 정보에 의존하므로, 사용자의 잠재적인 취향이나 다른 사용자와의 상호작용 정보를 반영하기는 어려움
- 협업 필터링:
  - 사용자 행동 기반 : 여러 사용자의 평가나 소비 데이터를 활용하여 사용자 간 혹은 아이템 간 유사도를 산출
  - 다양한 추천 : 콘텐츠 기반의 한계를 보완하여 사용자의 다양한 취향과 새로운 아이템을 추천 가능
  - 하이브리드 접근 : 콘텐츠 기반과 협업 필터링을 결합하여 더욱 정교한 추천 시스템을 구현할 수 있다(ex. Youtube)

- 취향이 비슷한 친구들에게 물어보는 것과 유사한 방식
- 사용자가 아이템에 매긴 평점 정보나 상품 구매 이력과 같은 사용자 행동 양식을 기반으로 추천을 수행하는 것

> 일반적으로 사용자 기반보다는 아이템 기반 정확도 더 높음
>- 비슷한 영화(또는 상품)을 좋아한다고 해서 사람의 취향이 비슷하다고 판단하기 어려운 경우 많음

In [3]:
import numpy as np
import pandas as pd
movies = pd.read_csv('data/movies.csv')
ratings = pd.read_csv('data/ratings.csv')

- 데이터셋 출처 : https://grouplens.org/datasets/movielens/latest/
- Grouplens 사이트에서 만든 MovieLens 데이터셋 활용

In [4]:
movies.info()
display(movies.head(3))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


- movieID : 영화 고유 ID, 각 영화의 식별자, 다른 데이터셋과 연결할 때 사용(PK)
- title : 영화 제목(출시년도)
- genres : 영화 장르 정보, | 기호로 구분되어 기록됨

In [5]:
ratings.info()
display(ratings.head(3))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224


- userId : 사용자의 고유 ID, 평점을 남긴 사용자 식별을 위해 사용
- movieId : 영화 고유 ID, movies 데이터셋의 movieId와 연결되어 영화 정보 참조에 활용(FK)
- rating : 사용자가 영화에 부여한 평점
- timestamp : 평점이 기록된 시각

In [6]:
# 추천에 필요없는 timestamp(평점이 기록된 시각) 제외
ratings = ratings.drop(columns = 'timestamp')
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


### 1. 사용자-아이템 평점 행렬 생성
- 1.1 사용자-아이템 평점 행렬 생성
- 1.2 결측치 채우

### 사용자-아이템 행렬:
<img src = "https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbtgDyd%2FbtryDKJCNjV%2FlMQwtLJBGuVEJhN28up0nK%2Fimg.png"/>

- 협업 필터링 기반은 최근접 이웃 방식과 잠재 요인 방식으로 나뉘는데, 두 방식 모두 사용자-아이템 행렬에 의지해서 추천을 수행

> **사용자-아이템 평점 행렬**
>- 각 행에는 사용자를, 각 열 아이템(영화, 상품 등)을 나타내며, 셀에는 해당 사용자가 해당 아이템에 대해 매긴 평점이나 구매, 클릭 등 상호작용 데이터가 기록된 행렬
>- NaN은 해당 아이템에 대한 평가가 없는 상태를 의미

In [7]:
ratings.columns

Index(['userId', 'movieId', 'rating'], dtype='object')

In [8]:
# 영화 제목 정보를 추가하기 위해 movies 데이터셋과 ratings 데이터셋을 movieId를 기준으로 병합
rating_movies = pd.merge(ratings, movies, on = 'movieId')
rating_movies.head(3)

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,Heat (1995),Action|Crime|Thriller


In [9]:
ratings_matrix = rating_movies.pivot_table(
    index = 'userId', # 행 : 사용자 아이디
    columns = 'title', # 열 : 영화 제목
    values = 'rating' # rkqt : 평점
)

display(ratings_matrix.head(3))
ratings_matrix.shape
# 행 : 610명의 고유 사용자 아이디
# 열 : 9719개의 고유 영화 제목
# ex) 1번 사용자가 "i Three Amigos! (1986)"를 보고 4.0을 부여함 (평점을 매기지 않은 경우 NaN)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(610, 9719)

- 1.2 결측치 채우기

In [10]:
# NaN 값은 해당 사용자가 해당 영화를 평가하지 않은 것이므로, 0으로 채워줌
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 2. 영화와 영화 간의 유사도 산출
- 2.1 아이템 기반을 위한 행렬 전치
- 2.2 영화 간(다중, 다중)의 코사인 유사도 계산

### 기억 기반(Memorial Based) 또는 최근접 이웃(Nearest Neighbour) 기반

> 1. 사용자 기반(User Based) : 사용자 a와 비슷한 사용자 b 이 아이템 사용함
> <img src = "https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FNXoXu%2FbtryM0yME48%2FBtJ37i5jNhmvWmwf7wOKsK%2Fimg.png"/>
> 1. 사용자 A와 B가 영화(또는 상품)에 대해 비슷한 평점을 주었다면,
> 2. 사용자 B가 높게 평점을 준 영화("프로메테우스")를 사용자 A에게도 추천해주는 방식
>- 장점 : "나와 비슷한 사람은 어떤 걸 좋아할까?"라는 관점이기 때문에, 사용자의 실제 취향을 직접 반영
>- 단점 : 사용자 수가 많아질수록 모든 사용자의 유사도를 계산하기가 어려움(확장성 문제)

### 2. 아이템 기반(Item Based) : 이 아이템을 구매한 다른 고객은 저 아이템도 구매함
<img src = "https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FIXchB%2FbtryLBTMHY9%2FKH9nCeQmb8k03aUDwScxp0%2Fimg.png"/>
> 1. 프로메테우스와 다크나이트가 유사한 평가 패턴을 가지고 있다면(거리가 가깝다면)
> 2. 사용자가 다크나이트를 보고 프로메테우스를 안 봤다면 프로메테우스를 추천 or 사용자가 프로메테우스를 좋아했다면 다크나이트를 추천해주는 방식
>- 장점 : 아이템(영화/상품) 간의 특성은 비교적 바뀌지 않아서, 사용자 기반보다 정확도가 높은 경우가 많음
>- 단점 : 새로운 아이템(신작 영화 등)에 대한 평가가 쌓이지 않으면, 유사도 계산이 어려움(콜드 스타트 문제)

- 2.1 아이템기반을 위한 행렬 전치

In [11]:
# T : Transpose(전치)
# 아이템 기반을 위한 "행:영화제목, 열:사용자 평점"으로 변경
ratings_matrix_T = ratings_matrix.T
ratings_matrix_T.head(3)

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- 2.2 영화 간(다중, 다중)의 코사인 유사도 계산

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
# 음악 추천에서는 (단일, 다중) -> 영화 추천에서는 (다중, 다중) 유사도 사용
item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)
item_sim.shape
item_sim[:3]
# ndarray 클래스는 컬럼명이 없어서 직관적으로 해석이 어려움

(9719, 9719)

In [14]:
ratings_matrix.columns
# ratings_matrix의 columns를 활용하여 DF화

Index([''71 (2014)', ''Hellboy': The Seeds of Creation (2004)',
       ''Round Midnight (1986)', ''Salem's Lot (2004)',
       ''Til There Was You (1997)', ''Tis the Season for Love (2015)',
       ''burbs, The (1989)', ''night Mother (1986)',
       '(500) Days of Summer (2009)', '*batteries not included (1987)',
       ...
       'Zulu (2013)', '[REC] (2007)', '[REC]² (2009)',
       '[REC]³ 3 Génesis (2012)',
       'anohana: The Flower We Saw That Day - The Movie (2013)',
       'eXistenZ (1999)', 'xXx (2002)', 'xXx: State of the Union (2005)',
       '¡Three Amigos! (1986)', 'À nous la liberté (Freedom for Us) (1931)'],
      dtype='object', name='title', length=9719)

In [15]:
# 영화 제목을 인덱스와 컬럼에 매핑
item_sim_df = pd.DataFrame(item_sim, index = ratings_matrix.columns, columns = ratings_matrix.columns)
item_sim_df.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),1.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.141653,0.0,...,0.0,0.342055,0.543305,0.707107,0.0,0.0,0.139431,0.327327,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,1.000000,0.707107,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Round Midnight (1986),0.0,0.707107,1.000000,0.0,0.0,0.0,0.176777,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0


In [17]:
# 예시 : "Avengers : Infinity War - Part I (2018)"
# 유사한 영화를 유사도 내림차순 정렬로 상위 5개를 출력
item_sim_df["Avengers: Infinity War - Part I (2018)"].sort_values(ascending = False)[1:6]

# [1:6] : 자기 자신(가장 높은 유사도) 제외
# 아이템 기반 유사도 데이터는 사용자의 평점 정보를 모두 취합해 영화에 따라 유사한 다른 영화를 추천하는 데 사용
# 유사도 점수가 높을수록 두 영화의 평가 패턴이 비슷함을 의미
# 대부분 어벤저스와 연관된 Marvel(또는 DC) 히어로물 영화

,Avengers: Infinity War - Part I (2018)
title,
Deadpool 2 (2018),0.802636
Thor: Ragnarok (2017),0.781017
Untitled Spider-Man Reboot (2017),0.696221
Justice League (2017),0.604227
Guardians of the Galaxy 2 (2017),0.591055


- 위에서 구한 아이템 기반 영화 유사도 데이터는 ``전체 사용자의 평점을 기준``으로 영화 간 유사도를 산출하여 영화 추천이 가능함
- 하지만, 영화 간의 ``단순 유사도``만 고려하기 때문에 ``보았던 작품``에 대해 중복되는 한계가 있다

### 3. 개인별로 아직 관람하지 않은 영화에 대해 예측 평점을 계산하고, 상위 영화를 추천
- 특정한 개인이 ``아직 관람하지 않은 영화``에 대해 ``아이템 유사도``와 ``기존 관람한 영화의 평점데이터``를 기반으로 하여 ``새롭게 모든 영화 예측 평점``을 수한 후 높은 예측 평점을 가진 영화를 추천하는 방식

#### 협업 필터링에서 자주 사용되는 가중 평균 방식 활용
-가중평균 : 자료의 평균을 구할 때 자료값의 중요도나 영향 정도에 해당하는 가중치를 반영하여 구한 평균값
- 아직 보지 않은 영화의 평점을 ``비슷한 영화의 평점``을 유사도로 가중치를 주어 평균내어 추정하는 방법 활용

> 계산 순서
> 1. 분자(Weighted Sum) : 각 이웃 아이템의 유사도x사용자가 매긴 평점을 모두 더함
> 2. 분모(Normalization) : 이웃 아이템의 유사도 절대값을 모두 더해 정규화 기준을 만듦
> Weighted Sum / Normalization => 예측 평점

- 가중 평균 방식

In [18]:
# 1) 테스트 대상 사용자와 영화 설정
user_id = 1 # 평점을 예측할 사용자 ID
k_neighbours = 3 # Top-k 이웃 영화 개수

In [19]:
# 2) 사용자의 평점 정보 가져오기
user_ratings = ratings_matrix.loc[user_id]
user_ratings # 1번 사용자가 매긴 모든 영화 평점(0은 미관람 영화)

,1
title,
'71 (2014),0.0
'Hellboy': The Seeds of Creation (2004),0.0
'Round Midnight (1986),0.0
'Salem's Lot (2004),0.0
'Til There Was You (1997),0.0
...,...
eXistenZ (1999),0.0
xXx (2002),0.0
xXx: State of the Union (2005),0.0


In [ ]:
# 예측 평점 대상 영화 선정

In [20]:
# 미관람 영화(평점 0점) 목록 조회 및 대상 영화 선택
unseen_items = user_ratings[user_ratings == 0].index
unseen_items[:5]

Index([''71 (2014)', ''Hellboy': The Seeds of Creation (2004)',
       ''Round Midnight (1986)', ''Salem's Lot (2004)',
       ''Til There Was You (1997)'],
      dtype='object', name='title')

In [21]:
# 미관람 영화인지 확인
"Avengers: Infinity War - Part I (2018)" in unseen_items

True

In [22]:
# 예측평점 대상영화로 설정
target_movie = "Avengers: Infinity War - Part I (2018)"

In [23]:
# 3) 대상 영화와 모든 영화 간 유사도 계산
movie_sim = item_sim_df[target_movie]
movie_sim

,Avengers: Infinity War - Part I (2018)
title,
'71 (2014),0.000000
'Hellboy': The Seeds of Creation (2004),0.000000
'Round Midnight (1986),0.000000
'Salem's Lot (2004),0.000000
'Til There Was You (1997),0.000000
...,...
eXistenZ (1999),0.000000
xXx (2002),0.143236
xXx: State of the Union (2005),0.000000


In [24]:
# 4) 사용자가 이미 본 영화 목록(watched)
list_movies = user_ratings[user_ratings > 0].index

# Series.nlargest(n) : 시리즈에서 값이 큰 순서대로 상위 n개의 원소를 반환
top_neighbours = movie_sim[list_movies].nlargest(k_neighbours) # 유사도 기준 상위 k개의 요소 반환
display(top_neighbours)

,Avengers: Infinity War - Part I (2018)
title,
Frankenstein (1931),0.227252
X-Men (2000),0.202147
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981),0.180971


### 분자 계산(Weighted Sum, Dot Product)
- 유사도X평점을 모두 더해 사용자 선호의 총합 점수를 구함
- 유사도가 높고, 평점이 높을 수록, 더 크게 기여
- 이 값은 ``사용자가 아직 보지 않은 영화에 대해 얼마나 좋아할지``를 합산한 점수
- 이 후 분모로 나누어 평균을 구하면, 예측 평점 도출 가능

ex) 친구 A, B, C의 추천력(유사도)과 만족도(평점)이 아래와 같다
- A : 유사도 0.8, 평점 4.0 -> 3.2
- B : 유사도 0.3, 평점 2.0 -> 0.6
- C : 유사도 0.9, 평점 5.0 -> 4.5

> 기여도 총합(Weighted Sum) : 8.3
> 분모 : 유사도의 총합(2.0)으로 나누어 계산
> 이 기여도/분모 나누어 4.15라는 평균 만족도를 얻을 수 있음

In [25]:
# 5) 분자 계산 (가중합 : 유사도 x 평점)
# np.dot() : 가중합 계산
weighted_sum = np.dot(top_neighbours, user_ratings[top_neighbours.index])
weighted_sum

np.float64(2.824598711174075)

In [26]:
# 6) 분모 계산(정규화 인자 : 유사도 절대값 합)
# - 모든 유사도 절대값을 합산해 예측 평점의 스케일을 조정(정규화 인자)
# - normalization으로 나누어 유사도 총합 차이 보정
normalization = top_neighbours.abs().sum()
normalization

np.float64(0.6103701660060481)

In [27]:
# 7) 예측 평점 계산
pred_rating = weighted_sum / normalization if normalization != 0 else 0
pred_rating # "Avengers: Infinity War - Part I (2018)"의 예측 평점

np.float64(4.627681476728806)

- 모든 미관람 영화에 대해 모든 예측평점 계산

In [28]:
from tqdm.auto import tqdm

predictions = {} # 영화제목 : 예측 평점

# 각 미관람 영화에 대해 순회
for movie in tqdm(unseen_items):
  # 1) 대상 영화와 모든 영화 간 유사도 조회
  movie_sim = item_sim_df[movie]

  # 2) 사용자가 평점 매긴 영화 목록
  list_movies = user_ratings[user_ratings > 0].index

  # 3) Top-k 이웃 영화 선택(유사도 기준)
  top_neighbours = movie_sim[list_movies].nlargest(k_neighbours)

  # 4) 분자 계산 (유사도x평점 합계)
  weighted_sum = np.dot(top_neighbours, user_ratings[top_neighbours.index])

  # 5) 분모 계산 (normalization 유사도 절대값 합)
  normalization = top_neighbours.abs().sum()

  # 6) 예측 평점
  pred_rating = weighted_sum / normalization if normalization != 0 else 0

  predictions[movie] = pred_rating

  0%|          | 0/9487 [00:00<?, ?it/s]

In [31]:
# predictoins Series or DataFrame 형태로 상위 20개 항목 출력
display(pd.Series(predictions).sort_values(ascending = False).head(20))

,0
The Brand New Testament (2015),5.0
Zazie dans le métro (1960),5.0
Big Hero 6 (2014),5.0
Big Fish (2003),5.0
Beyond the Sea (2004),5.0
"Big Blue, The (Grand bleu, Le) (1988)",5.0
"General, The (1926)",5.0
Son of God (2014),5.0
Million Dollar Baby (2004),5.0
Cyrano de Bergerac (1990),5.0
